In [1]:
!pip install transformers
!pip install torch

In [2]:
import pandas as pd

# Cargar y preparar los datos de entrenamiento desde el CSV
df = pd.read_excel("/content/salud_mental_preguntas_y_respuestas.xlsx")
texts = df["Pregunta"].tolist() + df["Respuesta"].tolist()  # Combina preguntas y respuestas
with open("corpus_salud_mental.txt", "w") as file:
    file.write("\n".join(texts))

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

tokenizer = GPT2Tokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = GPT2LMHeadModel.from_pretrained("datificate/gpt2-small-spanish")

In [4]:
!pip install transformers[torch] accelerate==0.20.1

In [5]:

# Cargar y preparar los datos de entrenamiento desde el CSV
#df = pd.read_csv("corpus_bienestar_mental.csv")
#texts = df["Pregunta"].tolist() + df["Respuesta"].tolist()  # Combina preguntas y respuestas
#with open("corpus_bienestar_mental.txt", "w") as file:
#    file.write("\n".join(texts))

# Tokenizar el corpus y crear el dataset
dataset = TextDataset(
    tokenizer=tokenizer,                  # Usar el tokenizador cargado anteriormente
    file_path="/content/corpus_salud_mental.txt",  # Ruta al archivo de datos de entrenamiento
    block_size=8,                        # Tamaño de bloque para dividir el corpus en secuencias
)

# Definir los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./chatbot_finetuned",    # Carpeta donde se guardarán los resultados del entrenamiento
    overwrite_output_dir=True,            # Sobrescribir la carpeta de salida si existe
    num_train_epochs=20,                 # Número de épocas de entrenamiento
    per_device_train_batch_size=128,     # Tamaño del lote por dispositivo durante el entrenamiento
)

# Entrenar el modelo
trainer = Trainer(
    model=model,                          # El modelo preentrenado que se va a afinar
    args=training_args,                   # Argumentos de entrenamiento configurados anteriormente
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),  # Formatear datos antes del entrenamiento
    train_dataset=dataset,                # Conjunto de datos de entrenamiento que se creó
)

trainer.train()  # Iniciar el proceso de entrenamiento

# Guardar el modelo entrenado
trainer.save_model("./chatbot_finetuned")  # Guardar el modelo en la carpeta especificada




/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


Step,Training Loss


In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = GPT2LMHeadModel.from_pretrained("chatbot_finetuned")  # Utiliza el directorio donde se guardo el modelo entrenado


In [19]:
# Definir la pregunta de entrada que deseas hacer al chatbot
input_text = "Pregunta: ¿Me siento mal, qué puedo hacer?"

# Tokenizar la pregunta y convertirla en IDs
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generar una respuesta basada en la pregunta de entrada
output = model.generate(input_ids, max_length=50, num_return_sequences=1, no_repeat_ngram_size=2)

# Decodificar la salida generada por el modelo en texto legible
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

# Imprimir la respuesta generada por el chatbot
print(generated_text)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Pregunta: ¿Me siento mal, qué puedo hacer?
¿Cómo puedo mejorar mi salud mental? Es importante consultar a un profesional para un diagnóstico y plan de tratamiento adecuados.
La salud emocional puede involucrar varias estrategias, como la terapia cognitivo


In [8]:
import shutil

shutil.make_archive("/content/chatbot_finetuned", 'zip', "/content/chatbot_finetuned")


'/content/chatbot_finetuned.zip'

In [9]:
from google.colab import files

files.download("/content/chatbot_finetuned.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>